In [1]:
# coding: utf-8
from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from compiler.ast import flatten
import math
import sys,random,os,cPickle
from sklearn.cross_validation import train_test_split
import matplotlib.pylab as plt
from sklearn import metrics
from pandas import DataFrame
from pandas import Series
from scipy.stats import mode

from sklearn.linear_model import LogisticRegression
from sklearn import metrics,cross_validation
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.externals import joblib
from sklearn.preprocessing import MinMaxScaler
xinshen_dir="D:\\fg_workspace\\python\\xinshen\\second\\new_data\\"
train=pd.read_csv(xinshen_dir+"xinshen_train_filled_na1.csv")
validate=pd.read_csv(xinshen_dir+"xinshen_validate_filled_na1.csv")
test=pd.read_csv(xinshen_dir+"xinshen_test_filled_na1.csv")

print 'the train data pos tag=%s,neg tag=%s'%(len(train[train['label']==1]),len(train[train['label']==0]))
print 'the validate data pos tag=%s,neg tag=%s'%(len(validate[validate['label']==1]),len(validate[validate['label']==0]))
print 'the test data pos tag=%s,neg tag=%s'%(len(test[test['label']==1]),len(test[test['label']==0]))


feat_imp=pd.read_csv(xinshen_dir+"train_features.csv")

D:\software\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: DeprecationWarning: The compiler package is deprecated and removed in Python 3.x.
D:\software\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


the train data pos tag=341090,neg tag=141725
the validate data pos tag=125636,neg tag=56210
the test data pos tag=67672,neg tag=41404


In [2]:
dh_train_ids=pd.read_csv(xinshen_dir+"dh_apply_ids_train.csv")
dh_validate_ids=pd.read_csv(xinshen_dir+"dh_apply_ids_validate.csv")
dh_test_ids=pd.read_csv(xinshen_dir+"dh_apply_ids_test.csv")

In [16]:
dh_train_list=dh_train_ids['apply_id'].tolist()
dh_validate_list=dh_validate_ids['apply_id'].tolist()
dh_test_list=dh_test_ids['apply_id'].tolist()

In [17]:
def isDhLabel(x,listA):
    if x in listA:
        return 1
    else:
        return 0

In [ ]:
train['is_dh']=train['apply_id'].apply(lambda x:isDhLabel(x,dh_train_list))

In [ ]:
train[['apply_id','is_dh','label']].head()

In [ ]:
validate['is_dh']=validate['apply_id'].apply(lambda x:isDhLabel(x,dh_validate_list))
test['is_dh']=test['apply_id'].apply(lambda x:isDhLabel(x,dh_test_list))

In [ ]:
train.head()

In [3]:
def get_month(x):
    return int(x.split("-")[1])

In [4]:
train['month']=train['time'].apply(lambda x:get_month(x))
validate['month']=validate['time'].apply(lambda x:get_month(x))
test['month']=test['time'].apply(lambda x:get_month(x))

In [5]:
print len(train)
print len(validate)
print len(test)

482815
181846
109076


In [6]:
print min(train['time']),max(train['time'])
print min(validate['time']),max(validate['time'])
print min(test['time']),max(test['time'])

2015-01-01 00:34:49 2015-12-31 18:06:23
2016-01-03 15:40:02 2016-06-30 17:11:09
2016-07-01 08:09:09 2016-09-30 16:59:56


In [13]:
new_train=train
new_validate=validate
new_test=test
dh_validate=pd.merge(validate,dh_validate_ids,how = 'inner',on='apply_id')
dh_test=pd.merge(test,dh_test_ids,how = 'inner',on='apply_id')

In [14]:
dh_validate_overdue=dh_validate#[dh_validate['label']==0]
dh_test_overdue=dh_test#[dh_test['label']==0]

In [15]:
print len(dh_validate_overdue)
print min(dh_validate_overdue['time']),max(dh_validate_overdue['time'])

128426
2016-01-03 15:40:02 2016-06-30 17:11:09


In [20]:
# test_sub=test[test['month']==9]

In [7]:
train.drop(['time','apply_id','bs_apply_id','transport_id','mortgagor_id','month'],axis=1,inplace=True)
validate.drop(['time','apply_id','bs_apply_id','transport_id','mortgagor_id','month'],axis=1,inplace=True)

In [8]:
test.drop(['time','apply_id','bs_apply_id','transport_id','mortgagor_id','month'],axis=1,inplace=True)

In [ ]:
dh_validate_overdue.drop(['time','apply_id','bs_apply_id','transport_id','mortgagor_id','month'],axis=1,inplace=True)
dh_test_overdue.drop(['time','apply_id','bs_apply_id','transport_id','mortgagor_id','month'],axis=1,inplace=True)

In [9]:
feat_imp.head(40)

,coef,columns
0,95.965002,default_times
1,24.025560,month_income
2,16.098480,n1
3,14.299903,month_payment
4,7.245070,mean
5,6.773414,n2
6,6.643913,contact_num
7,5.773904,consumption_habits
8,5.524379,product_type
9,5.490790,pledge_num


In [10]:
selected_features=feat_imp[feat_imp['coef']>0]['columns'].tolist()[:20]
print selected_features

['default_times', 'month_income', 'n1', 'month_payment', 'mean', 'n2', 'contact_num', 'consumption_habits', 'product_type', 'pledge_num', 'latest_month_income', 'nomal_card_num', 'accept_moth_repay', 'loan_max_num', 'max_loan_age', 'nomal_card_used_amount', 'fee_months', 'max_credit_card_age', 'applying_count', 'risk_industry']


WOE变量 ：暂时选择consumption_habits  product_type   risk_industry

In [11]:
print train['consumption_habits'].value_counts().index.tolist()
print validate['consumption_habits'].value_counts().index.tolist()
print test['consumption_habits'].value_counts().index.tolist()

[1.0, -1.0, 0.0, 2.0, 3.0]
[-1.0, 1.0, 0.0, 2.0]
[-1.0, 1.0, 0.0, 2.0, 3.0]


In [21]:
print train['default_times'].value_counts()

-1      260340
 0      145981
 1       71590
 2        4473
 3         379
 4          26
 5          13
 6           4
 450         1
 420         1
 401         1
 310         1
 188         1
 182         1
 485         1
 110         1
 120         1
Name: default_times, dtype: int64


In [20]:
print np.log(0)

-inf


In [16]:
print validate['risk_industry'].value_counts()

 0    67346
 1    66750
-1    47750
Name: risk_industry, dtype: int64


In [17]:
print test['risk_industry'].value_counts()

-1    38794
 0    35479
 1    34803
Name: risk_industry, dtype: int64


In [40]:
def generate_woe_indicator(x,bad,good):
    x0=x[0]
    x1=x[1]
    return np.log((x0*1.0/x1)/(bad*1.0/good))

In [43]:
# c_train_df=train['product_type'].groupby([train['consumption_habits'],train['label']]).count()
c_train_df=pd.crosstab(train.consumption_habits,train.label,margins=True)
c_train_df.reset_index(inplace=True)
c_train_df.drop(["All"],axis=1,inplace=True)
c_train_df.columns=['consumption_habits','bad','good']
c_total_bad=c_train_df[c_train_df['consumption_habits']=='All']['bad'].tolist()[0]
c_total_good=c_train_df[c_train_df['consumption_habits']=='All']['good'].tolist()[0]
c_train_df=c_train_df[c_train_df['consumption_habits']!='All']
c_train_df['consumption_habits_woe']=c_train_df[['bad','good']].apply(lambda x:generate_woe_indicator(x,c_total_bad,c_total_good),axis=1)

p_train_df=pd.crosstab(train.product_type,train.label,margins=True)
p_train_df.reset_index(inplace=True)
p_train_df.drop(["All"],axis=1,inplace=True)
p_train_df.columns=['product_type','bad','good']
p_total_bad=p_train_df[p_train_df['product_type']=='All']['bad'].tolist()[0]
p_total_good=p_train_df[p_train_df['product_type']=='All']['good'].tolist()[0]
p_train_df=p_train_df[p_train_df['product_type']!='All']
p_train_df['product_type_woe']=p_train_df[['bad','good']].apply(lambda x:generate_woe_indicator(x,p_total_bad,p_total_good),axis=1)

r_train_df=pd.crosstab(train.risk_industry,train.label,margins=True)
r_train_df.reset_index(inplace=True)
r_train_df.drop(["All"],axis=1,inplace=True)
r_train_df.columns=['risk_industry','bad','good']
r_total_bad=r_train_df[r_train_df['risk_industry']=='All']['bad'].tolist()[0]
r_total_good=r_train_df[r_train_df['risk_industry']=='All']['good'].tolist()[0]
r_train_df=r_train_df[r_train_df['risk_industry']!='All']
r_train_df['risk_industry_woe']=r_train_df[['bad','good']].apply(lambda x:generate_woe_indicator(x,r_total_bad,r_total_good),axis=1)

In [44]:
print c_train_df
print p_train_df
print r_train_df

  consumption_habits    bad    good  consumption_habits_woe
0                 -1  95490   68993                1.203274
1                  0  25858   56595                0.094957
2                  1  20315  214690               -1.479578
3                  2     62     811               -1.692876
4                  3      0       1                    -inf
  product_type    bad    good  product_type_woe
0            1  86123  154904          0.291229
1            3  34718  144278         -0.546226
2            5      0      16              -inf
3            6      6      77         -1.673788
4            7  20878   41807          0.183890
5           28      0       8              -inf
  risk_industry    bad    good  risk_industry_woe
0            -1  73017      11           9.678810
1             0  33886  176456          -0.771812
2             1  34822  164623          -0.675151


In [45]:
def log_binary(x):
    return np.log(x)

In [50]:
woe_features=['consumption_habits','product_type','risk_industry']

In [46]:
print selected_features

['default_times', 'month_income', 'n1', 'month_payment', 'mean', 'n2', 'contact_num', 'consumption_habits', 'product_type', 'pledge_num', 'latest_month_income', 'nomal_card_num', 'accept_moth_repay', 'loan_max_num', 'max_loan_age', 'nomal_card_used_amount', 'fee_months', 'max_credit_card_age', 'applying_count', 'risk_industry']


In [47]:
for column_name in selected_features:
    new_name=column_name+"_log"
    train[new_name]=train[column_name].apply(lambda x:log_binary(x))

In [51]:
train=pd.get_dummies(train, columns=woe_features)
print train.head()

   label  city_id  domicile_province  domicile_city  resident_province  \
0      1   320500             320000         320900             320000   
1      1   410100             410000         410100             410000   
2      1   150400             150000         150400             150000   
3      1   370300             370000         371600             370000   
4      0   350200             350000         350200             350000   

   resident_city  identity_address_province  identity_address_city  \
0         320500                     320000                 320500   
1         410100                     410000                 410100   
2         150400                     150000                 150400   
3         371600                     370000                 371600   
4         350200                     350000                 350200   

   apply_max_amount  long_repayment_term        ...          \
0             60000                   24        ...           
1       

In [52]:
for column_name in selected_features:
    new_name=column_name+"_log"
    validate[new_name]=validate[column_name].apply(lambda x:log_binary(x))
    test[new_name]=test[column_name].apply(lambda x:log_binary(x))
validate=pd.get_dummies(validate, columns=woe_features)
test=pd.get_dummies(test, columns=woe_features)
    

In [55]:
test['product_type'].value_counts()

KeyError: 'product_type'

In [33]:
train_df.head()

product_type  city_id  domicile_province  \
consumption_habits label                                             
-1                 0             95490    95490              95490   
                   1             68993    68993              68993   
 0                 0             25858    25858              25858   
                   1             56595    56595              56595   
 1                 0             20315    20315              20315   

                          domicile_city  resident_province  resident_city  \
consumption_habits label                                                    
-1                 0              95490              95490          95490   
                   1              68993              68993          68993   
 0                 0              25858              25858          25858   
                   1              56595              56595          56595   
 1                 0              20315              20315          20315   

                          identity_address_province  identity_address_city  \
consumption_habits label                                                     
-1                 0                          95490                  95490   
                   1                          68993                  68993   
 0                 0                          25858                  25858   
                   1                          56595                  56595   
 1                 0                          20315                  20315   

                          apply_max_amount  long_repayment_term      ...       \
consumption_habits label                                             ...        
-1                 0                 95490                95490      ...        
                   1                 68993                68993      ...        
 0                 0                 25858                25858      ...        
                   1                 56595                56595      ...        
 1                 0                 20315                20315      ...        

                          sum_current_month_bill_amount  sum_overdue_sum  \
consumption_habits label                                                   
-1                 0                              95490            95490   
                   1                              68993            68993   
 0                 0                              25858            25858   
                   1                              56595            56595   
 1                 0                              20315            20315   

                          amount  organization_portion  is_overdue  \
consumption_habits label                                             
-1                 0       95490                 95490       95490   
                   1       68993                 68993       68993   
 0                 0       25858                 25858       25858   
                   1       56595                 56595       56595   
 1                 0       20315                 20315       20315   

                          max_renege_count_24  sum_overdue_180  \
consumption_habits label                                         
-1                 0                    95490            95490   
                   1                    68993            68993   
 0                 0                    25858            25858   
                   1                    56595            56595   
 1                 0                    20315            20315   

                          channel_source  known_creditease  is_city_same  
consumption_habits label                                                  
-1                 0               95490             95490         95490  
                   1               68993             68993         68993  
 0                 0               25858             25858         25858  
                  

In [27]:
train['risk_industry'].value_counts()

 0    210342
 1    199445
-1     73028
Name: risk_industry, dtype: int64

In [21]:
train[features].head()  

,default_times,month_income,n1,month_payment,mean,n2,contact_num,consumption_habits,product_type,pledge_num,latest_month_income,nomal_card_num,accept_moth_repay,loan_max_num,max_loan_age,nomal_card_used_amount,fee_months,max_credit_card_age,applying_count,risk_industry
0,2,7462.57,7459.93,190.96,3731.285000,7459.93,6,1,1,-1,7467.85,3,5000,-1,-1,10430,9,95,1,1
1,0,3867.03,3867.03,0.00,1933.515000,3867.03,6,1,1,-1,0.00,2,3000,-1,-1,17482,0,4,1,1
2,-1,4714.18,0.00,0.00,2357.091667,4535.55,6,1,3,-1,4828.00,2,1000,24,24,40769,0,20,1,0
3,-1,4233.56,3459.90,0.00,2116.780000,4115.46,6,1,1,-1,5125.32,1,3000,-1,-1,0,0,15,1,1
4,2,1885.33,0.00,0.00,942.666667,3186.00,6,1,1,-1,0.00,2,2000,54,54,27033,0,80,1,1


In [16]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
features=selected_features###list(set(set(train.columns.tolist())-set(['label'])))#,'is_dh'
print 'label' in features
train_x=scaler.fit_transform(train[features].values)  
train_y=train['label'].values

# val_x=scaler.transform(validate_sub[features].values)
# val_y=validate_sub['label'].values

# test_x=scaler.transform(test[features].values)
# test_y=test['label'].values

False


In [17]:
val_x=scaler.transform(validate[features].values)
val_y=validate['label'].values

In [18]:
test_x=scaler.transform(test[features].values)
test_y=test['label'].values

In [16]:
dh_validate_overdue_x=scaler.transform(dh_validate_overdue[features].values)
dh_validate_overdue_y=dh_validate_overdue['label'].values

In [17]:
dh_test_overdue_x=scaler.transform(dh_test_overdue[features].values)
dh_test_overdue_y=dh_test_overdue['label'].values

In [19]:
from sklearn.externals import joblib
lr = LogisticRegression(penalty='l2',C=10.0,class_weight='balanced',random_state=1024,solver='sag')
lr.fit(train_x,train_y)
joblib.dump(lr,xinshen_dir+'lr_model_selected_features.pkl') 
lr = joblib.load(xinshen_dir+'lr_model_selected_features.pkl') 

In [20]:
val_probs=lr.predict_proba(val_x)
val_auc=roc_auc_score(val_y,val_probs[:,1])
print 'val set auc is=%s'%(val_auc)###原始的auc:0.931777552037
# print pd.DataFrame({"columns":list(train[features].columns), "coef":list(lr.coef_.T)})
# print "***************************"
# test_probs=clf.fit(train_x,train_y).predict_proba(test_x)
# test_auc=roc_auc_score(test_y,test_probs[:,1])
# print 'test set auc is=%s'%(test_auc)

val set auc is=0.893247029387


In [15]:
test_probs=lr.predict_proba(test_x)
test_auc=roc_auc_score(test_y,test_probs[:,1])
print 'test set auc is=%s'%(test_auc)##原始的auc:0.967835776337
train_features_df=pd.DataFrame({"columns":list(train[features].columns), "coef":list(lr.coef_.T)})
train_features_df.to_csv(xinshen_dir+"train_features_imp_5_11.csv",index=False)
print train_features_df

test set auc is=0.967835776337
                    coef                        columns
0      [-0.593722337496]                    query_times
1     [-0.0512668665896]                   is_city_same
2      [-0.165487525035]                        city_id
3      [-0.693941663147]                 overdue_num_c2
4       [-1.06389573416]                 overdue_num_c1
5      [-0.131350472358]                 overdue_num_c6
6        [-0.7063889903]                 overdue_num_c5
7       [-7.34869812886]                   refuse_times
8                  [0.0]                  borrower_type
9     [-0.0330612271308]                house_condition
10      [0.590990182229]               known_creditease
11       [0.18606794976]  sum_current_month_bill_amount
12      [0.239380066911]           organization_portion
13       [1.82716420716]                  risk_industry
14        [3.1851209454]                   loan_max_num
15      [0.294912799829]                 channel_source
16      [0.524665

In [ ]:
#贷后相关信息的统计
val_probs_list=val_probs[:,1].tolist()
test_probs_list=test_probs[:,1].tolist()
val_auc_df=DataFrame({'label':validate['label'].tolist(),'apply_id':validate['apply_id'].tolist(),'auc':val_probs_list,'is_dh':validate['is_dh'].tolist()})

In [ ]:
test_auc_df=DataFrame({'label':test['label'].tolist(),'apply_id':test['apply_id'].tolist(),'auc':test_probs_list,'is_dh':test['is_dh'].tolist()})
test_auc_df.head()

In [29]:
dh_validate_auc_df=pd.merge(val_auc_df,dh_validate_ids,how='inner',on='apply_id')
dh_test_auc_df=pd.merge(test_auc_df,dh_test_ids,how='inner',on='apply_id')
print len(dh_validate_auc_df),len(dh_validate_ids)
print len(dh_test_auc_df),len(dh_test_ids)

128426 128425
67688 67681


In [30]:
dh_validate_auc_df.head()

,apply_id,auc,label
0,5580845,0.983233,1
1,5580989,0.985032,1
2,5587308,0.975511,1
3,5594189,0.972975,1
4,5594844,0.964305,1


In [48]:
val_auc_df.head()

,apply_id,auc,is_dh,label
0,5580845,0.983233,1,1
1,5580989,0.985032,1,1
2,5587308,0.975511,1,1
3,5587982,0.000002,0,0
4,5594189,0.972975,1,1


In [50]:
ndh_validate_auc_matrix=val_auc_df[val_auc_df['is_dh']==0].values
total_cnt=0
for i in range(len(ndh_validate_auc_matrix)):
#     if ndh_validate_auc_matrix[i][1]>=0.5 and ndh_validate_auc_matrix[i][2]==1:
#         cnt=1
    if ndh_validate_auc_matrix[i][1]<0.5 and ndh_validate_auc_matrix[i][2]==0 :
        cnt=1
    else:
        cnt=0
    total_cnt+=cnt
print total_cnt*1.0/len(val_auc_df[val_auc_df['label']==0]) 

0.950364703789


In [32]:
dh_validate_auc_matrix=dh_validate_auc_df.values
total_cnt=0
for i in range(len(dh_validate_auc_matrix)):
#     if dh_validate_auc_matrix[i][1]>=0.5 and dh_validate_auc_matrix[i][2]==1:
#         cnt=1
    if dh_validate_auc_matrix[i][1]<0.5 and dh_validate_auc_matrix[i][2]==0 :
        cnt=1
    else:
        cnt=0
    total_cnt+=cnt
print total_cnt*1.0/len(dh_validate_auc_df[dh_validate_auc_df['label']==0]) 

0.0111111111111


In [42]:
validate.head()

,label,time,apply_id,bs_apply_id,transport_id,mortgagor_id,product_type,city_id,domicile_province,domicile_city,...,sum_current_month_bill_amount,sum_overdue_sum,amount,organization_portion,is_overdue,max_renege_count_24,sum_overdue_180,channel_source,known_creditease,is_city_same
0,1,2016-01-04 16:20:40,5580845,5580845,14720289,14718798,1,330300,330304,330300,...,-1,-1,-1,-1,1,0,0,-1,6,1
1,1,2016-01-06 16:22:57,5580989,5580989,14731770,14730279,1,130300,130302,130300,...,-1,-1,-1,-1,1,0,0,-1,3,1
2,1,2016-01-05 14:48:01,5587308,5587308,14724580,14723089,1,110100,-1,130600,...,-1,-1,-1,-1,0,0,0,-1,3,1
3,0,2016-01-04 12:22:10,5587982,5587982,14717931,14716440,1,210200,-1,210200,...,-1,-1,-1,-1,0,0,0,-1,6,1
4,1,2016-01-05 09:21:54,5594189,5594189,14721565,14720074,3,150400,152502,152500,...,-1,-1,-1,-1,0,0,0,-1,3,0


In [40]:
val_auc_df.head()

,apply_id,auc,label
0,5580845,0.983233,1
1,5580989,0.985032,1
2,5587308,0.975511,1
3,5587982,0.000002,0
4,5594189,0.972975,1


In [41]:
val_auc_df_matrix=val_auc_df.values
total_cnt=0
for i in range(len(val_auc_df_matrix)):
#     if dh_validate_auc_matrix[i][1]>=0.5 and dh_validate_auc_matrix[i][2]==1:
#         cnt=1
    if val_auc_df_matrix[i][1]<0.5 and val_auc_df_matrix[i][2]==0 :
        cnt=1
    else:
        cnt=0
    total_cnt+=cnt
print total_cnt*1.0/len(val_auc_df[val_auc_df['label']==0]) 

0.950916207081


# bad case分析

In [33]:
dh_validate_auc_df.head()

,apply_id,auc,label
0,5580845,0.983233,1
1,5580989,0.985032,1
2,5587308,0.975511,1
3,5594189,0.972975,1
4,5594844,0.964305,1


In [38]:
all_dh_validate_auc_df=dh_validate_auc_df[dh_validate_auc_df['label']==0]
bad_case_dh_vlidate_df=all_dh_validate_auc_df[all_dh_validate_auc_df['auc']>0.5]
bad_case_dh_vlidate_df.drop(['label'],axis=1,inplace=True)
bad_case_dh_vlidate=pd.merge(bad_case_dh_vlidate_df,validate,how='inner',on='apply_id')
bad_case_dh_vlidate.to_csv(xinshen_dir+"bad_case_dh_vlidate.csv",index=False)

D:\software\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [39]:
train.head()

,label,time,apply_id,bs_apply_id,transport_id,mortgagor_id,product_type,city_id,domicile_province,domicile_city,...,sum_current_month_bill_amount,sum_overdue_sum,amount,organization_portion,is_overdue,max_renege_count_24,sum_overdue_180,channel_source,known_creditease,is_city_same
0,1,2015-01-05 11:00:57,3529839,3529839,12886140,12897781,1,320500,320000,320900,...,-1,-1,-1,-1,1,0,0,-1,-1,1
1,1,2015-01-07 20:11:31,3594350,3594350,12905161,12916821,1,410100,410000,410100,...,-1,-1,-1,-1,0,0,0,-1,-1,1
2,1,2015-01-15 10:33:43,3594687,3594687,12951132,12962837,3,150400,150000,150400,...,-1,-1,-1,-1,1,0,0,-1,-1,1
3,1,2015-01-06 14:57:38,3605031,3605031,12895032,12906681,1,370300,370000,371600,...,-1,-1,-1,-1,1,0,0,-1,-1,0
4,0,2015-02-09 18:49:24,3618351,3618351,13140203,13144373,1,350200,350000,350200,...,-1,-1,-1,-1,1,0,0,-1,-1,1


In [ ]:
# dh_validate_probs=lr.predict_proba(dh_validate_overdue_x)
dh_validate_auc=roc_auc_score(dh_validate_auc_df['label'].values,dh_validate_auc_df['auc'].values)
print 'dh_validate_auc set auc is=%s'%(dh_validate_auc)

In [ ]:
# dh_test_probs=lr.predict_proba(dh_test_overdue_x)
dh_test_auc=roc_auc_score(dh_test_auc_df['label'].values,dh_test_auc_df['auc'].values)
print 'dh_test_auc set auc is=%s'%(dh_test_auc)

In [16]:
train_features_df=pd.read_csv(xinshen_dir+"train_features_imp_5_11.csv")

train_features_df.sort(columns=['coef'],ascending=False,inplace=True)
train_features_df.to_csv(xinshen_dir+"train_features_imp_5_11.csv",index=False)

D:\software\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


In [16]:
train_features_df=pd.DataFrame({"columns":list(train[features].columns), "coef":list(lr.coef_.T)})
train_features_df.to_csv(xinshen_dir+"train_features_imp.csv",index=False)
print train_features_df

                   coef                        columns
0     [-0.600726984173]                    query_times
1    [-0.0483623112044]                   is_city_same
2     [-0.131551845285]                        city_id
3     [-0.874782962283]                 overdue_num_c2
4     [-0.988285398033]                 overdue_num_c1
5    [-0.0505852944786]                 overdue_num_c6
6     [-0.455938002956]                 overdue_num_c5
7      [-6.95303297045]                   refuse_times
8                 [0.0]                  borrower_type
9    [-0.0534968448051]                house_condition
10     [-1.26017408467]               known_creditease
11     [0.150465590601]  sum_current_month_bill_amount
12     [0.221833850212]           organization_portion
13      [1.70780541264]                  risk_industry
14      [3.75750945478]                   loan_max_num
15      [1.59311141134]                 channel_source
16     [0.489940523033]        overdue_loan_num90_days
17    [-0.

In [21]:
train['month_income'].value_counts()

-1.00         161385
 0.00            565
 2000.00         341
 3000.00         323
 3500.00         214
 2500.00         187
 1500.00         166
 1800.00         124
 2200.00         123
 5000.00         115
 2800.00         108
 2300.00         108
 2400.00          97
 4000.00          95
 2600.00          93
 2100.00          91
 1600.00          83
 2666.67          79
 1900.00          78
 1200.00          77
 1000.00          71
 1700.00          71
 3100.00          69
 2333.33          68
 3333.33          68
 3400.00          65
 6000.00          61
 3200.00          60
 2900.00          60
 1400.00          59
               ...  
 2756.06           1
 8994.58           1
 6494.39           1
 9889.51           1
 33028.33          1
 2118.38           1
 3817.15           1
 9438.67           1
 4431.92           1
 2602.56           1
 26354.00          1
 2646.46           1
 2500.27           1
 8217.70           1
 4852.29           1
 4228.77           1
 2588.45     

In [22]:
validate['month_income'].value_counts()

-1.0000         92393
 0.0000           933
 3000.0000        121
 2000.0000        107
 3500.0000         99
 2500.0000         64
 2300.0000         45
 2600.0000         45
 2200.0000         44
 2100.0000         42
 2400.0000         37
 2800.0000         33
 2700.0000         29
 3200.0000         28
 3300.0000         28
 3985.0000         24
 2150.0000         23
 3150.0000         22
 2250.0000         21
 2625.0000         21
 2750.0000         21
 2900.0000         20
 3250.0000         20
 2350.0000         19
 4955.0000         19
 2050.0000         18
 2650.0000         17
 3450.0000         17
 2880.0000         17
 2825.0000         17
                ...  
 5180.0050          1
 2821.4600          1
 5547.6292          1
 6063.9400          1
 4692.3500          1
 5375.3375          1
 1915.7350          1
 3013.8100          1
 3518.3425          1
 138637.0300        1
 4750.7325          1
 3911.4625          1
 6294.2750          1
 3110.4850          1
 9834.0125

In [16]:
#####以验证集作为测试集

In [17]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
features_v=list(set(set(validate.columns.tolist())-set(['label'])))
val_x=scaler.fit_transform(validate[features_v].values)  
val_y=validate['label'].values

test_x=scaler.transform(test[features_v].values)
test_y=test['label'].values

from sklearn.externals import joblib
lrv = LogisticRegression(penalty='l2',C=10.0,class_weight='balanced',random_state=1024,solver='sag')
lrv.fit(val_x,val_y)
joblib.dump(lrv,xinshen_dir+'lrv_model.pkl') 
lrv = joblib.load(xinshen_dir+'lrv_model.pkl')

testv_probs=lrv.predict_proba(test_x)
testv_auc=roc_auc_score(test_y,testv_probs[:,1])
print 'test set auc is=%s'%(testv_auc)

val_features_df=pd.DataFrame({"columns":list(validate[features_v].columns), "coef":list(lrv.coef_.T)})
val_features_df.to_csv(xinshen_dir+"val_features_imp.csv",index=False)
print val_features_df

test set auc is=0.809644735805
                   coef                        columns
0       [4.95425650745]                    query_times
1     [-0.013319550232]                   is_city_same
2     [-0.382665308322]                        city_id
3    [-0.0782373871618]                 overdue_num_c2
4      [0.429988183653]                 overdue_num_c1
5      [0.130749643297]                 overdue_num_c6
6     [-0.929336198527]                 overdue_num_c5
7      [-29.3394554762]                   refuse_times
8      [-4.74498486826]              stop_pay_card_num
9     [-0.048870751741]                house_condition
10    [-0.062226500821]               known_creditease
11     [-6.18203183501]  sum_current_month_bill_amount
12     [-1.69201270901]           organization_portion
13      [1.96944447555]                  risk_industry
14      [6.03681504563]                   loan_max_num
15    [-0.765158887925]                 channel_source
16    [-0.516237478361]        ove

D:\software\Anaconda2\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [19]:
train_features_df=pd.read_csv(xinshen_dir+"train_features_imp.csv")
val_features_df=pd.read_csv(xinshen_dir+"val_features_imp.csv")

train_features_df.sort(columns=['coef'],ascending=False,inplace=True)
val_features_df.sort(columns=['coef'],ascending=False,inplace=True)

train_features_df.to_csv(xinshen_dir+"train_features_imp.csv",index=False)
val_features_df.to_csv(xinshen_dir+"val_features_imp.csv",index=False)

D:\software\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
D:\software\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [15]:
feat_df=DataFrame({'feat':features})
feat_df.to_csv(xinshen_dir+"features.csv",index=False)